<div style="border:solid Chocolate 2px; padding: 40px">

<b>Привет!👋</b>

Меня зовут Огурлиев Анзор, я буду ревьюером твоего проекта. Сразу хочу предложить тебе общаться на "ты", если тебе это неудобно, скажи мне об этом. Нам предстоит пройти с тобой через тернистый и от того интересный путь анализа данных! Можешь не переживать, я всегда помогу тебе, если у тебя возникнут вопросы)

По ходу твоей работы я буду оставлять свои комментарии, по выполненным тобой шагам. Попрошу тебя не удалять их, так мне будет проще отслеживать твой прогресс в случае повторной проверки.

Очень важно вести диалог и обсуждать те пункты, которые у тебя могли не получиться, поэтому, ты можешь оставлять свои комментарии, и мы обязательно во всем разберемся! Самое главное - выделяй свои комментарии удобным для тебя способом (цвет, рамка), но так, чтобы они отчетлитво отличались от моих комментариев. Так мы ни в чем не запутаемся)

Свои комментарии я буду выделять так: <font color='green'>зеленый</font>, <font color='gold'>желтый</font> и <font color='red'>красный</font> цвет, например:

<br/>

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Этот комментарий я буду оставлять, если твое решение оптимально для данного шага.
</div>

<br/>

<div class="alert alert-warning">
    <h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Некоторые замечания и рекомендации💡:</b> Это не означает, что ты совершил ошибку. Такими комментариями я буду отмечать места, где ты справился с задачей, но её можно решить ещё лучше. Также в таких комментариях я буду оставлять для тебя другие советы.
</div>


<br/>
<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
Если решение, которое ты применил на определенном шаге, требует значительных изменений, то я буду оставлять такой комментарий. Тебе нужно будет подумать еще раз как правильным образом решить данный этап. Если ревью содержит такие комментарии, то я не смогу принять проект с первого раза.
</div>

P.S. В оглавлении комментария ты увидишь `V1` (`V1`, `V2`, `V3`) - это версия моего комментария, которая понадобится нам с тобой, чтобы не запутаться, если потребуется оставить тебе комментарии после первой проверки.

# Сборный проект по первому блоку курса Data Science

# 1. Введение

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

### Описание данных
- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Молодец, что начинаешь проект с описания!

# 2. Открытие файла с данными и изучение общей информации

In [ ]:
import pandas as pd
import math
from scipy import stats as st
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
!pip install missingno
import missingno as msno
import seaborn as sns

sns.set()

In [ ]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')
display(df.head())
df.info()

Источник данных представляет собой таблицу из 11 колонок и 16715 строк. Содержание колонок в целом соответствует описанию данных из Введения.

<div class="alert alert-warning">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Некоторые замечания и рекомендации💡:</b> Можно было еще построить гистограммы распределений и составить краткое описание данных: количество пропусков, типы данных и тд.

# 3. Подготовка данных

## 3.1 Замена названий столбцов (приведение к нижнему регистру)

In [ ]:
df.columns = map(lambda x:x.lower(),df.columns)
df.head()

<div class="alert alert-warning">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Некоторые замечания и рекомендации💡:</b> Можно было проще привести названия к нижнему регистру: 
    
`data.columns = data.columns.str.lower()`

<div style="border:dashed green 1px; background:#097482; padding: 20px">
<span style="color:white">Мои комментарии V1:</span>

Так по пропускам есть отдельный пункт ниже я там все и описал

Названия столбцов переведены в нижний регистр

## 3.2 Преобразование данных в нужные типы

In [ ]:
def is_whole_number(number):
    return math.floor(number) == number
def all_whole_numbers(numbers):
    return all(map(is_whole_number, numbers))
df_temp = df['critic_score'].dropna()
print(all_whole_numbers(df_temp))

Проверил что в колонке **critic_score** все валидные значения не имеют дробной части и предсатвляют собой целое число процентов c нулем после десятичной точки.

In [ ]:
print('Количество записей в таблице с значенем user_score == tbd: ', df[df['user_score'] == 'tbd']['user_score'].count())
print('Количество записей в таблице с значенем user_score == 0: ', df[df['user_score'] == '0']['user_score'].count())
df.loc[df['user_score'] == 'tbd','user_score'] = '0'
print('Количество записей в таблице с значенем user_score == tbd: ', df[df['user_score'] == 'tbd']['user_score'].count())
print('Количество записей в таблице с значенем user_score == 0: ', df[df['user_score'] == '0']['user_score'].count())


Посчитал количество записей в таблице с значенем user_score равным tbd и 0

In [ ]:
df.info()
display(df.head())
for col in ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']:
    df[col] = df[col] * 1000000
df = df.astype({"name":'string',"platform":'string',"year_of_release":'Int64',"genre":'string',"na_sales":'int',"eu_sales":'int',"jp_sales":'int',"other_sales":'int',"critic_score":'Int64',"rating":'string'}, errors = 'ignore')
df['user_score'] = pd.to_numeric(df['user_score'], errors = 'coerce')
df.info()
df.head()

Были проведены следующие изменения типов:
- колонки **name** , **platform**, **genre**, **rating** переводятся в **string** так как они содержат строковые значения (или NaN)
- колонка **year_of_release** содержит год релиза который представляет из себя целое число, я использовал приведение к dtype Int64, при котором произошло и отбрасывание дробной части в виде 0
- колонки **na_sales**, **eu_sales**, **jp_sales**, **other_sales** содержат число миллионов с дробной частью - я перевел сначала их просто в число продаж путем умножения на 1 000 000 а потом привел к целому - так как использование чисел типа float в Python сопряжено с накопленной ошибкой вычисления и по возможности если можно их надо избегать. В данном случае именно такой случай.
- колонка **critic_score** содержит рейтинг от критиков в интервале от 0 до 100, который не содержит дробной части после запятой (выше я провел проверку этого факта). Таким образом, эта колонка содержит целое число процентов и ее тип можно перевести в int с отбрасыванием дробной части
Для преобразования типов я использовал режим **errors = 'ignore'** который в случае ошибки оставлял исходное значения - в даном случае все NaN были заменены на <NA> которые далее могут быть обнаружены и удалены в случе необходимости
    
Если сравнить выводы **df.info()** до и после преобразования типов, то можно увидеть, что в некоторых колонках нет пропусков и число значений - общему числу строк = 16715. Для ряда колонок число значений меньше и это число не изменилось после изменения типов колонки - за исключением колонки **user_score** где помимо значений NaN пристутсвует еще и ряд строк с строковым значеним = tbd, которое не может быть преобразовано в float и таких строк довольно большое количество = 2424 то есть их нежелательно просто отбросить. 
Колонка **user_score** содержит рейтинг пользователя в интервале от 0 до 10, который может содержать дробную часть - поэтому я ее тип надо перевести в float, используя метод to_numeric, но для того, чтобы избежать ошибок парсинга пердварительно я заменю все значения **tbd** на строковое значение 0 а потом преобразованием типа перевел его в 0 типа float. При этом я подсчитал количество записей с изначальным значением **user_score** == 0 и это оно = 1 - то есть можно эти данные проигнорировать и "слить" их в одну группу.

## 3.3 Обработка пропусков

In [ ]:
msno.bar(df);
msno.matrix(df);
#df_null = df.iloc[:, [i for i, n in enumerate(np.var(df.isnull(), axis = 'rows')) if n > 0]]
df_with_nulls = df[['critic_score', 'user_score', 'rating']].isnull()
display(df_with_nulls.corr())
df['rating'] = df['rating'].fillna('No rating')

- Матрица корреляции для колонок **critic_score**, **user_score** и **rating**
- Столбчатая диаграмма для пропусков по всем колонкам
- Матрица пропущенных значений для всех колонок

### 3.3.1 Объясните, почему заполнили пропуски определённым образом или почему не стали это делать

Общее количество строк в dataframe равно 16715 но ряд столбцов имеет пропуски и меньшее число значимых величин а именно:
- name - 2 пропуска
- year_of_release - 269 пропусков
- genre - 2 пропуска
- critic_score - 8578 пропусков
- user_score - 6701 пропусков и 2424 значения **tbd** которые не могут быть проеобразованы в float и они были установлены в 0
- rating - 6766 пропусков
    
Таким образом, для колонок **critic_score**, **user_score** и **rating** количество пропусков довольно большое порядка 50%.
Матрица пропущенных значений и матрица корреляции для пропущенных значений в этих колонок показывают, что корреляция пропущенных значений между полями **user_score** и **rating** близка к 1, корреляция по другим полям близка к 0.78 что указывает на определенную линейную корреляцию что говорит о том, что наличие пропусков по этим трем колонкам может быть взаимосвязано. Я предлагаю оставить пропуски для колонок **critic_score**, **user_score** где содержатся количественные величины как есть - то есть в рассчетах статистических характеристик они не будут принимать участия а установив их например в 0 или в медиану для всего распределения мы можем исказить результаты рассчетов там, где фигурируют эти параметры. Для колонки **rating** (c категориальной величиной) я бы устнановил значение всех ячеек NaN в какое то нейтральное строковое значени - например в строку 'No rating', которое не соответствует ни одному из домена существующих значений и мы могли бы видеть все эти пропущенные значения, собранные на гистограмме.
    
Пропуски я оставил после преобразования типов как есть (NaN или \<NA\>), значение 'tbd' для колонки **user_score** установил в 0 (подробно описал в предыдущем пункте 3.2)

### 3.3.2 Опишите причины, которые могли привести к пропускам
- Для полей **name**, **genre** - ошибки ввода - например при заполнении формы. Предлагаю их оставить как есть так как их количество очень мало и ими можно пренебречь при рассчете характеристик.
- Поле **year_of_release** изначально было числовым и там исключено наличие нечисловых данных - пропуск может быть связан либо с ошибкой при вводе даных - например формат ввода даты не соответствовал предложенному шаблону. 
- Пропуски в полях **critic_score**, **user_score**, **rating** могут быть связаны либо с ошибкой при вводе данных, либо по каким то играм не был проставлен рейтинг по каким то причинам (но заменять пропуски на значение 0 все равно нежелательно).

### 3.3.3 Обратите внимание на аббревиатуру 'tbd' в столбце с оценкой пользователей. Отдельно разберите это значение и опишите, как его обработать
Уже объяснено в п 3.2

<div class="alert alert-warning">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Некоторые замечания и рекомендации💡:</b> 
    
Молодец, что так подробно все анализируешь и описываешь! Можно было сразу удалить игры без названия и платформы в принципе, они никак не анализ не повлияют, зато датафрейм будет больше очищен от возможных ошибок. Также и игры без года релиза можно было сразу удалить.
    
Все другие параметры обработал абсолютно верно!

## 3.4 Bar-диаграммы по некоторым столбцам

In [ ]:
ax = df['platform'].value_counts().plot(kind="bar", title='Распределение по платформам', figsize=(7, 4), xlabel='Название платформы', ylabel='Частота использования')
ax.yaxis.grid(True)
plt.show()


ax = df['rating'].value_counts().plot(kind="bar", title='Распределение по рейтингу', figsize=(7, 4), xlabel='Рейтинг', ylabel='Частота использования')
ax.yaxis.grid(True)
#ax.bar_label(ax.containers[0], label_type='edge')
plt.show()

ax = df['genre'].value_counts().plot(kind="bar", title='Распределение по жанру', figsize=(7, 4), xlabel='Жанр', ylabel='Частота использования')
ax.yaxis.grid(True)
#ax.bar_label(ax.containers[0], label_type='edge')
plt.show()


<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
Старайся, чтобы библиотеки у тебя локально были той же версии, что и на сервере, выше ошибка возникла из-за разных версий библиотек. Ты как аналитик должен это предусматривать всегда.

Я привел распределения по следующим полям в виде bar-диаграмм (в задание не входило - сделал для понимания):
- название платформы
- рейтинг
- жанр

По платформам top5 это
- PS2
- DS
- PS3
- Wii
- X360

По жанрам top5 это
- Action
- Sports
- Misc
- Role-Playing
- Shooter

По рейтингу - самый часто встречающийся рейтинг E, рейтинги E-C, K-A, RP, AO крайне редки. Вариант **No rating** занимает порядка 40% от всего количества данных

## 3.5 Подсчет суммарных продаж во всех регионах

In [ ]:
df['all_sales'] = df['na_sales'] +  df['eu_sales'] +  df['jp_sales'] +  df['other_sales']
df.head()

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Суммарные продажи посчитаны верно.

# 4. Провести исследовательский анализ данных

## 4.1 Распределение выпусков игр по годам

In [ ]:
df_games_by_years = df.groupby(by='year_of_release')['name'].count()

ax = df_games_by_years.plot(kind="bar", title='Количество релизов игр по годам', figsize=(7, 4), xlabel='Год', ylabel='Количество релизов')
ax.yaxis.grid(True)
#ax.bar_label(ax.containers[0], label_type='edge')
plt.show()

Диаграмма распределения количества релизов игр по годам имеет колоколообразную форму с максимумом в районе 2008-2009 г. Для значений года <= 1994 значения количества релизов становится слишком малым (< 1%) и этими данными можно пренебречь.

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Именно так, все верно.

## 4.2 Распределение продаж по платформам

In [ ]:
def plot_sub_plots(df, list_top):
    for platform in list_top:
        df_temp = df[df['platform'] == platform].groupby(by='year_of_release')['all_sales'].sum()
        ax = df_temp.plot(kind="bar", title=f'Выручка по годам для {platform}', figsize=(4, 3), xlabel='Год', ylabel='Выручка')
        current_values = plt.gca().get_yticks()
        ax.set_yticks(ax.get_yticks().tolist())
        plt.gca().set_yticklabels(['{:,}'.format(x) for x in current_values])
        plt.show()

In [ ]:
df_sells_by_platforms = df.groupby(by='platform')['all_sales'].sum()
df_sells_by_platforms = df_sells_by_platforms.sort_values()
ax = df_sells_by_platforms.plot(kind="bar", title='Распределение продаж по платформам за весь период наблюдений', figsize=(7, 4), xlabel='Платформа', ylabel='Сумма продаж')
ax.yaxis.grid(True)
plt.show()

top5platforms_list = list(df_sells_by_platforms.index[-5:])
top5platforms_list.reverse()
print('Top 5 список платформ с максимальными продажами за весь период наблюдений: ', top5platforms_list)
plot_sub_plots(df, top5platforms_list)


Из анализа распределения количества релизов по годам видно, что для данных, расопложенных ранее чем 1994 год, количество релизов очень мало (1994 год составляет менее 1 % от максимума) - поэтому далее ниже этой границы можно отбросить.
Данные по годовым распределеням по top5 платформ, отсортированных по суммарной выручке, показали, что средний срок жизни платформы 10 лет, и у каждой из платформ за последние 3-4 года происходит резкий спад продаж - во всей видимости это связано с выходом новых платформ.

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Абсолютно верный вывод! Платформы живут около 10 лет.

## 4.3 Данные за актуальный период

In [ ]:
display(df['year_of_release'].value_counts().sort_index().tail())
df_actual_period = df[df['year_of_release'] >= 2014].copy()
df_actual_period.info()

Актуальный период был выбран = 3 годам так как этот период характеризует среднюю длительность спада или подъема на рынке продаж игр на конкретной платформе. Таким образом, создан dataframe **df_actual_period**, в котором собраны данные только за три последних года - то есть за годы 2014, 2015, 2016

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Актуальный период подобран правильно, так держать!

## 4.4 Выбор потенциально прибыльных платформ

In [ ]:
platform_list = list(df_actual_period['platform'].value_counts().index)
plot_sub_plots(df_actual_period, platform_list)

Анализ прибыльности платформ в выбранном актуальном периоде [2014, 2016] показал, что к прибыльным платформам за этот период можно отнести:
- PS4
- XOne
- 3DS

Для первых двух есть максимум в 2015 г, для 3DS убывающий тренд но не такой сильных как для остальных платформ. И эти данные можно объяснить тем, что PS4 и XOne это самые современные на текущий период времени платформы.

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Верно, прежде всего стоит обратить внимание именно на эти платформы.

## 4.5 Построение графика «ящик с усами» по глобальным продажам игр в разбивке по платформам

In [ ]:
def plot_sub_plots_box(df, topylim=0):
    plt.figure(figsize=(15,8))
    box_plot = sns.boxplot(data=df, x="platform", y="all_sales");
    box_plot.set(xlabel='Платформа',ylabel='Выручка')
    #box_plot.set_yticks(ax.get_yticks().tolist())
    current_values = plt.gca().get_yticks()
    plt.gca().set_yticklabels(['{:,}'.format(x) for x in current_values])
    plt.title("Графики boxplot для продаж, сгруппированными по платформам")
    plt.ylim(bottom=0)
    if topylim != 0:
        plt.ylim(top=topylim)
    plt.show()

In [ ]:
display(df_actual_period.head())

In [ ]:
df_count_by_platform = df_actual_period.groupby(by='platform')['all_sales'].count()
df_count_by_platform = df_count_by_platform.sort_values()
ax = df_count_by_platform.plot(kind="bar", title='Распределение количества продаж по платформам', figsize=(7, 4), xlabel='Платформа', ylabel='Количество продаж')
ax.yaxis.grid(True)
plt.show()
df_count_by_platform = df_actual_period.groupby(by='platform')['all_sales'].sum()
df_count_by_platform = df_count_by_platform.sort_values()
ax = df_count_by_platform.plot(kind="bar", title='Распределение суммы продаж по платформам', figsize=(7, 4), xlabel='Платформа', ylabel='Сумма продаж')
ax.yaxis.grid(True)
plt.show()
plot_sub_plots_box(df_actual_period)
plot_sub_plots_box(df_actual_period, 800000)


<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
Нужно построить 1 график, на котором будет находиться несколько боксплотов. Воспользуйся библиотекой seaborn, она позволит легко такой график построить. На одном графике будет гораздо легче сравнивать значения статистик разных платформ.
    
Метод ylim() попробуй по другому использовать, укажи верхнюю и нижную границу, сейчас у тебя некоторые графики не читаемы из-за масштаба, нельзя понять какое значение принимает медиана или квартиль. 

Анализ графиков Boxplot для платформ, которые попали в dataframe с актуальным периодом показывает, что
- минимальное количество выбросов для платформ Wii, PSP - но для них и малое количество продаж
- большое количество выбросов для платформ PS4, PCV, XOne с большим количеством продаж

Я построил графики Boxplot в разбивке продаж по платформам сначала общий, потом его же с увеличенной областью Медиана ± квартили и можно сделать следующие выводы:

- Из графиков видно, что лидер по количеству и сумме продаж - PS4 - имеет самые большие выбросы "усов" и второй по величие выброс квартилей 25-75 что объяснимо большим количеством и суммой продаж. При этом, какие либо игры могут продаваться за более высокую цену ввиду своей высокой популярности и активной пердпродажной рекламной кампании и я бы не назвал это выбросами, от которых надо избавляться
- странная картина по графику boxplot для платформы Wii - самый большой размах квартилей - хотя сама платформа в аутсайдерах и по количеству и по сумме продаж. Возможно, это связано с недостаточно репрезентативной выборкой.
- медианная выручка для большей части платформ = 50000000, есть меньшие значения 20000000 для 3DS и PC, для остальных плафторм выборка не вляется репрезентативной.
- если взять платформы из top3 по сумме продаж - **PS4, 3DS, XOne** - то прямоугольник Медиана ± Квартили для 3DS смещен вниз по сравнения с PS4 и XOne. Я это могу объяснить тем, что 3DS - это карманная игровая приставка с мини-экраном, игры для нее стоят в среднем меньше чем для стационарных приставок, таких как PS4 и XOne 

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
Помимо выбросов сравни статистики платформы, мдеианы и квартили, что они означают, как значения статистик интерпретировать?

## 4.6 Анализ влияния на продажи внутри одной популярной платформы отзывов пользователей и критиков

In [ ]:
df_for_ps4 = df_actual_period.loc[df_actual_period['platform'] == 'PS4', ['all_sales', 'critic_score', 'user_score']]
display(df_for_ps4.corr())
df_for_ps4.plot.scatter(x='critic_score', y='all_sales');
df_for_ps4.plot.scatter(x='user_score', y='all_sales');

Построена матрица корреляции между суммарными продажами, отзывами пользователей и критиков а также диаграмма рассеяния 
- между продажами и отзывами критиков есть слабая линейная положительная корреляция
- между продажами и отзывами пользователей корреляции нет 

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Именно так, все верно.

## 4.7 Сравнение выводов с продажами игр на других платформах

In [ ]:
def display_corr_matrix(df, col_list):
    for platform in col_list:
        print(f'******** Матрица корреляций для платформы {platform} *********')
        df_corr = df.loc[df['platform'] == platform, ['all_sales', 'critic_score', 'user_score']]
        display(df_corr)
        display(df_corr.corr())
        print()

def plot_sub_plots_scatter(df, col_list):
    for platform in col_list:
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,8))
        df_for_platform = df_actual_period.loc[df_actual_period['platform'] == platform, ['all_sales', 'critic_score', 'user_score']]
        df_for_platform.plot.scatter(x='critic_score', y='all_sales', ax=axes[0], title= f'Диаграмма рассеяния для {platform}', figsize=(7,3));
        df_for_platform.plot.scatter(x='user_score', y='all_sales', ax=axes[1], title= f'Диаграмма рассеяния для {platform}');
        plt.show()

In [ ]:
display_corr_matrix(df_actual_period, platform_list)
plot_sub_plots_scatter(df_actual_period, platform_list)

Построены матрицы корреляции и диаграммы рассеяния между колонками **(all_sales, critic_score), (all_sales, user_score)** для всех платформ из актуального периода, по результатам можно сделать следующие выводы:
- если говорить о платформах из top3 по сумме продаж - **PS4, 3DS, XOne** то корреляция между рейтингом критиков и продажами для PS4 и XOne выше, чем для 3DS
- корреляция между рейтингом пользователей и продажами для 3DS выше, чем для PS4, XOne
- в целом я не вижу среди коэффициентов корреляций значений выше, чем 0.525723 - это коэффициент недостаточный для того, чтобы говорить о сильной корреляционной зависимости
- значения NaN в корреляционной матрице для некоторых платформ связаны с тем, что в соответствующих сериях нет значений, отличных от NaN

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
Посчитай также коэффициенты корреляции.

Ты для PS4 написал, что от оценок пользователей зависимости нет, а оценки критиков коррелируют с продажами, на других платформах схожие зависимости не прослеживаются?

## 4.8 Анализ общего распределения игр по жанрам

In [ ]:
df_sells_by_genres = df_actual_period.groupby(by='genre')['all_sales'].count()
df_sells_by_genres = df_sells_by_genres.sort_values()
ax = df_sells_by_genres.plot(kind="bar", title='Распределение количества продаж по жанрам', figsize=(7, 4), xlabel='Жанр', ylabel='Количество продаж')
current_values = plt.gca().get_yticks()
ax.set_yticks(ax.get_yticks().tolist())
plt.gca().set_yticklabels(['{:,}'.format(x) for x in current_values])
plt.ylim(bottom=0)
plt.show()

df_sells_by_genres = df_actual_period.groupby(by='genre')['all_sales'].sum()
df_sells_by_genres = df_sells_by_genres.sort_values()
ax = df_sells_by_genres.plot(kind="bar", title='Распределение суммы продаж по жанрам', figsize=(7, 4), xlabel='Жанр', ylabel='Сумма продаж')
current_values = plt.gca().get_yticks()
ax.set_yticks(ax.get_yticks().tolist())
plt.gca().set_yticklabels(['{:,}'.format(x) for x in current_values])
plt.ylim(bottom=0)
plt.show()

df_sells_by_genres = df_actual_period.groupby(by='genre')['all_sales'].median()
df_sells_by_genres = df_sells_by_genres.sort_values()
ax = df_sells_by_genres.plot(kind="bar", title='Распределение медианы продаж по жанрам', figsize=(7, 4), xlabel='Жанр', ylabel='Медиана продаж')
current_values = plt.gca().get_yticks()
ax.set_yticks(ax.get_yticks().tolist())
plt.gca().set_yticklabels(['{:,}'.format(x) for x in current_values])
plt.ylim(bottom=0)
plt.show()

df_sells_by_genres = df_actual_period.groupby(by='genre')['all_sales'].median()
df_sells_by_genres = df_sells_by_genres.sort_values()
ax = df_sells_by_genres.plot(kind="bar", title='Распределение среднего продаж по жанрам', figsize=(7, 4), xlabel='Жанр', ylabel='Среднее продаж')
current_values = plt.gca().get_yticks()
ax.set_yticks(ax.get_yticks().tolist())
plt.gca().set_yticklabels(['{:,}'.format(x) for x in current_values])
plt.ylim(bottom=0)
plt.show()

Анализ диаграммы распределения продаж по жанрам показывает, что:
- самый прибыльный жанр это **Action** - по нему самое большое количество и сумма продаж
- однако если смотреть на распределение медианных и средних продаж - то лидер там это жанр **Shooter** (±500000) а **Action** имеет медиану примерно в 5 раз меньше. Это говорит о том, что характерная продажа в жанре Shooter приносит больше прибыли но жанр Action берет свое за счет количества продаж.
- существенной разницы между медианным и средним распределением нет что говорит об отсутствии больших выбросов
- top3 самых прибыльных жанров по суммарным продажам это:
    - Action
    - Shooter
    - Sports
- top3 самых прибыльных жанров по медианным продажам это:
    - Shooter
    - Sports
    - Platform
- жанры Puzzle и Strategy пользуются наименьшей популярностью

<div class="alert alert-block alert-danger">
<h2> Комментарий ревьюера V1<a class="tocSkip"></h2>


<b>На доработку🤔:</b>
В данном пункте помиму суммарных продаж нужно проанализировать средние или медианные продажи. На суммарные продажи большое влияние могут оказывать продажи игр бестселлеров, из-за них общие продажи могут быть максимальными, а вот продажи на одну игру значительно ниже.

# 5. Составление портрета пользователя каждого региона

## 5.1 Определение самых популярных платформ (top 5)

In [ ]:
def plot_sub_plots_platforms_by_region(df, region_list):
    for region in region_list:
        print(f'top 5 лидеров платформ по продажам для региона {region}')
        df_for_region = df_actual_period.groupby(by='platform')[region].sum()
        df_for_region = df_for_region.sort_values()
        print(df_for_region.tail(5))
        df_for_region.tail(5).plot(kind='pie', labels=None, label='', autopct='%1.0f%%', legend=True, figsize=(6,6), title=f'top 5 лидеров платформ по продажам для региона {region}');
        plt.show()

In [ ]:
plot_sub_plots_platforms_by_region(df_actual_period, ['na_sales', 'eu_sales', 'jp_sales'])

<div class="alert alert-warning">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Некоторые замечания и рекомендации💡:</b> Здесь можно было бы использовать метод subplots(), чтобы построить графики в одном полотне, например, в одном ряду, чтобы они все на одном экране сразу поместились и можно было легко их сравнить.

Анализ top-5 платформ по продажам в регионах показал:
- для Европы и Северной Америки 1е и 2е места по продажам - PS4 и XOne
- для Японии 1е место 3DS, 2e место - PS4
- самую большую долю от продаж 1е место в топе занимает PS4 в Европе - 55%
- в остальных двух регионах 1е место в топе занимает меньше 50% продаж

## 5.2 Определение самых популярных жанров (top 5)

In [ ]:
def plot_sub_plots_genres_by_region(df, region_list):
    for region in region_list:
        print(f'top 5 лидеров жанров по продажам для региона {region}')
        df_for_region = df_actual_period.groupby(by='genre')[region].sum()
        df_for_region = df_for_region.sort_values()
        print(df_for_region.tail(5))
        df_for_region.tail(5).plot(kind='pie', labels=None, label='', autopct='%1.0f%%', legend=True, figsize=(6,6), title=f'top 5 лидеров жанров по продажам для региона {region}');
        plt.show()

In [ ]:
plot_sub_plots_genres_by_region(df_actual_period, ['na_sales', 'eu_sales', 'jp_sales'])

Анализ top-5 жанров по продажам в регионах показал:
- первые два места в top 5:
    - Европа - Action, Shooter
    - Северная Америка - Shooter, Action
    - Япония - Role-Playing, Action
- жанр на первом месте занимает от 30 до 40 %
- жанр на втором месте ненамного меньше в процентном отношении чем первый - 22-29%
- остальные жанры занимают намного меньшие доли по продажам во всех регионах

## 5.3 Влияние рейтинга ESRB на продажи в отдельном регионе

In [ ]:
def plot_sub_plots_rating_by_region(df, region_list):
    for region in region_list:
        df_for_region = df_actual_period.groupby(by='rating')[region].sum()
        df_for_region = df_for_region.sort_values()
        df_for_region.plot(kind='pie', labels=None, label='', autopct='%1.0f%%', legend=True, figsize=(6,6), title=f'Распределение продаж по рейтингу ESRB для региона {region}');
        plt.show()

In [ ]:
plot_sub_plots_rating_by_region(df_actual_period, ['na_sales', 'eu_sales', 'jp_sales'])

Анализ распределения продаж по рейтингу ESRB для регионов показал:
- для Европы и Северной Америки самый популярный рейтинг M - Mature 17+ - Для возраста более 17 лет
- для Европы и Северной Америки порядок рейтингов одинаковый - отличаются ненамного лишь сами значения
- для Японии самый популярный рейтинг T - Teens - от 13 лет и выше но также очень большую долю занимают значения No rating - именно те, на которые я заменил пропуски в этой колонке. Порядок рейтингов отличается от Европы и Северной Америки

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Портреты пользователей составил абсолютно верно, продолжай в том же духе!

# 6. Проверка гипотез

## 6.1 Средние пользовательские рейтинги платформ Xbox One и PC одинаковые

In [ ]:
alpha = 0.05 # уровень статистической значимости
# если p-value окажется меньше него, отвергнем гипотезу

df_urating_xone = df_actual_period[df_actual_period['platform'] == 'XOne']['user_score']
df_urating_pc = df_actual_period[df_actual_period['platform'] == 'PC']['user_score']
results = st.ttest_ind(df_urating_xone, df_urating_pc, nan_policy='omit')
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

- Нулевая гипотеза - средние пользовательские рейтинги платформ XOne и PC одинаковые
- Альтернативная гипотеза - средние пользовательские рейтинги платформ XOne и PC разные

Принимаем уровень статистической значимости alpha = 0.05
### Результат
Рассчитаное значение p-value для данных выбророк = 0.5858 и больше, чем принятый уровень статистической значимости - то есть нулевая гипотеза не отвергнута. Таким образом, средние пользовательские рейтинги платформ Xbox One и PC одинаковые в пределах данной статистической значимости.

## 6.2 Средние пользовательские рейтинги жанров Action и Sports разные

In [ ]:
alpha = 0.05 # уровень статистической значимости
# если p-value окажется меньше него, отвергнем гипотезу

df_urating_action = df_actual_period[df_actual_period['genre'] == 'Action']['user_score']
df_urating_sports = df_actual_period[df_actual_period['genre'] == 'Sports']['user_score']
results = st.ttest_ind(df_urating_action, df_urating_sports, nan_policy='omit')
print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

- Нулевая гипотеза - средние пользовательские рейтинги жанров Action и Sports одинаковые
- Альтернативная гипотеза - средние пользовательские рейтинги жанров Action и Sports разные

Принимаем уровень статистической значимости alpha = 0.05
### Результат
Рассчитаное значение p-value для данных выбророк = 9.92015750638991e-06 и меньше, чем принятый уровень статистической значимости - то есть нулевая гипотеза отвергнута. Таким образом, принята альтернативная гипотеза и средние пользовательские рейтинги жанров Action и Sports разные в пределах данной статистической значимости.

<div class="alert alert-success">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Правильно сформулировал и проверил гипотезы, молодец!

# 7. Выводы

- Распределение выпусков игр по годам за весь период наблюдений показывает, что до 1984 г количество выпусков игр было небольшим (так как технологии производства приставок только начинались), максимум приходится на 2008-2009 годы и к 2016 году небольшой спад
- Top 5 список платформ с максимальными продажами за весь период наблюдений:
    - PS2
    - X360
    - PS3
    - Wii
    - DS
- Анализ этого списка показал, что средний срок жизни платформы 10 лет, и у каждой из платформ за последние 3-4 года происходит резкий спад продаж - во всей видимости это связано с выходом новых платформ.
- В качестве актуального периода выбраны три последние года, за которые происходили наблюдения: 2014, 2015, 2016 так как он отражает наиболее актуальные данные по самым современным платформам и именно их надо использовать для планирования стратегии на 2017г
- Анализ прибыльности платформ в выбранном актуальном периоде [2014, 2016] показал, что к прибыльным платформам за этот период можно отнести:
    - PS4
    - XOne
    - 3DS
- Анализ графиков Boxplot для платформ, которые попали в dataframe с актуальным периодом показывает, что
    - минимальное количество выбросов для платформ **Wii, PSP** - но для них и малое количество продаж
    - большое количество выбросов для платформ **PS4, PCV, XOne** с большим количеством продаж
    - Из графиков видно, что лидер по количеству и сумме продаж - PS4 - имеет самые большие выбросы "усов" и второй по величие выброс квартилей 25-75 что объяснимо большим количеством и суммой продаж. При этом, какие либо игры могут продаваться за более высокую цену ввиду своей высокой популярности и активной пердпродажной рекламной кампании и я бы не назвал это выбросами, от которых надо избавляться
    - странная картина по графику boxplot для платформы Wii - самый большой размах квартилей - хотя сама платформа в аутсайдерах и по количеству и по сумме продаж. Возможно, это связано с недостаточно репрезентативной выборкой.
    - медианная выручка для большей части платформ = 50000000, есть меньшие значения 20000000 для 3DS и PC, для остальных плафторм выборка не вляется репрезентативной.
    - если взять платформы из top3 по сумме продаж - **PS4, 3DS, XOne** - то прямоугольник Медиана ± Квартили для 3DS смещен вниз по сравнения с PS4 и XOne. Я это могу объяснить тем, что 3DS - это карманная игровая приставка с мини-экраном, игры для нее стоят в среднем меньше чем для стационарных приставок, таких как PS4 и XOne 
- Построены матрицы корреляции и диаграммы рассеяния между колонками продажи, рейтинг пользователей, рейтинг критиков для всех платформ из актуального периода, по результатам можно сделать следующие выводы:
    - если говорить о платформах из top3 по сумме продаж - **PS4, 3DS, XOne** то корреляция между рейтингом критиков и продажами для PS4 и XOne выше, чем для 3DS
    - корреляция между рейтингом пользователей и продажами для 3DS выше, чем для PS4, XOne
    - в целом я не вижу среди коэффициентов корреляций значений выше, чем 0.525723 - это коэффициент недостаточный для того, чтобы говорить о сильной корреляционной зависимости
    - значения NaN в корреляционной матрице для некоторых платформ связаны с тем, что в соответствующих сериях нет значений, отличных от NaN
- Анализ диаграммы распределений жанрам показывает, что:
    - самый прибыльный жанр это **Action** - по нему самое большое количество и сумма продаж
    - однако если смотреть на распределение медианных и средних продаж - то лидер там это жанр **Shooter** (±500000) а **Action** имеет медиану примерно в 5 раз меньше. Это говорит о том, что характерная продажа в жанре Shooter приносит больше прибыли но жанр Action берет свое за счет количества продаж.
    - существенной разницы между медианным и средним распределением нет что говорит об отсутствии больших выбросов
    - top3 самых прибыльных жанров по суммарным продажам это:
        - Action
        - Shooter
        - Sports
    - top3 самых прибыльных жанров по медианным продажам это:
        - Shooter
        - Sports
        - Platform
    - жанры Puzzle и Strategy пользуются наименьшей популярностью
- Анализ top-5 платформ по продажам в регионах показал:
    - для Европы и Северной Америки 1е и 2е места по продажам - PS4 и XOne
    - для Японии 1е место 3DS, 2e место - PS4
    - самую большую долю от продаж 1е место в топе занимает PS4 в Европе - 55%
    - в остальных двух регионах 1е место в топе занимает меньше 50% продаж
- Анализ top-5 жанров по продажам в регионах показал:
    - первые два места в top 5:
        - Европа - Action, Shooter
        - Северная Америка - Shooter, Action
        - Япония - Role-Playing, Action
    - жанр на первом месте занимает от 30 до 40 %
    - жанр на втором месте ненамного меньше в процентном отношении чем первый - 22-29%
    - остальные жанры занимают намного меньшие доли по продажам во всех регионах
- Анализ распределения продаж по рейтингу ESRB для регионов показал:
    - для Европы и Северной Америки самый популярный рейтинг M - Mature 17+ - Для возраста более 17 лет
    - для Европы и Северной Америки порядок рейтингов одинаковый - отличаются ненамного лишь сами значения
    - для Японии самый популярный рейтинг T - Teens - от 13 лет и выше но также очень большую долю занимают значения No rating - именно те, на которые я заменил пропуски в этой колонке. Порядок рейтингов отличается от Европы и Северной Америки
- Проверка гипотез
    - Средние пользовательские рейтинги платформ Xbox One и PC одинаковые - на самом деле доля продаж игр на платформе XBox существенно выше чем доля продаж для PC как в целов так и по отдельным регионам
    - Средние пользовательские рейтинги жанров Action и Sports разные - но тем не менее продажи по этим жанрам в топе и по значениям близки друг к другу
    - Из этого можно сделать вывод, что полагаться на зависимость продаж от пользовательского рейтинга нельзя

#### Рекомендации по планированию торговой политики на 2017 г
- рекомендуется публиковать для продажи больше игр для платформ PS4, XOne, 3DS
- преимущество делать для игр в жанрах Action, Shooter, Role-Playing
- и отдавать преимущество играм с рейтингом ESRB M (Mature 17+) и T (Teens 13+)
- если говорить о политике распределения по регионам то рекомендации следующие:
    - Европа и Северная Америка - эти два рынка очень похожи
        - преимущество для платформ PS4, XOne
        - преимущество для жанров Action, Shooter
        - преимущество для рейтинга ESRB M
    - Япония
        - преимущество для платформ 3DS, PS4
        - преимущество для жанров Role-Playing, Action
        - преимущество для рейтинга ESRB T
- рекомендации для последующих шагов:
    - увеличивать выборку для данных с небольшим количеством - так чтобы можно было рассчитывать например корреляционные характеристики
    - при сборе данных с колтчественными характеристиками избегать большого количества пропусков
    - для более улучшенной характеристики зависимости продаж от рейтнгов критиков и пользователей построить зависимости медианы, количества и суммы продаж от диапазона рейтингов, разбитых на категории с помощью методов **cut** и **qcut**

<div class="alert alert-warning">
<h2> Комментарий ревьюера V1<a class="tocSkip"> </h2>

<b>Некоторые замечания и рекомендации💡:</b> В итоговом выводе нужно достаточно подробно написать основные пункты выявленных зависимостей. Стоит доработать вывод с учетом моих комментариев по ходу работы. В итоговом выводе также старайся оставлять рекомендации для последующих шагов. Например рекомендации, что можно обработать и проанализировать другие признаки, на что твоим коллегам стоить обратить внимание.

<div style="border:dashed green 1px; background:#097482; padding: 20px">
<span style="color:white">Мои комментарии version:</span>

Я хотел бы привести тут формат оформления ноутбука, который я буду использовать:

-  Введение
-  Шаги проекта оформленные как markdown header 1го уровня - например **2.Предобработка данных**
-  Внутри каждого шага промежуточные задачи как заголовки 2го уровня (если они есть)- например 
   - **2.1 Приведите столбец date к типу даты pandas**
   - **2.2 Создайте новый столбец с номером месяца на основе столбца date**
   - **2.3 Проверьте наличие пропущенных значений и дубликатов в датафреймах.** 
- Сначала идет заголовок 1го уровня в ячейке Markdown, потом - если есть - заголовок второго уровня в ячейке Markdown, потом код на Python, потом описание этого шага в ячейке Markdown
- Выводы - включают в себя результаты по каждому пункту и общий вывод в виде рекомендаций для бизнеса.

Все свои комментарии в ответ на комментарии ревьюера я буду далее добавлять, используя этот стиль

<div style="border:solid Chocolate 2px; padding: 40px">

**Общий вывод по проекту**

Спасибо за твой проект! Я уверен, что ты приложил все усилия, чтобы сделать этот проект. Это видно по твоей работе, а сформулированные выводы отражают, насколько успешно ты осваиваешь материал.

**Отмечу отдельные положительные моменты проекта🙂**:

* Ты полностью следовал всем этапам выполнения задания и справился почти со всем. Хорошая работа!
* Отдельно выделю некоторые выводы, которые ты смог сформулировать абсолютно точно!
* У тебя опрятный и читаемый код. Возможно это звучит банально, однако при работе с чужим кодом это крайне важно)


**На что стоит обратить внимание🤓**:

* Важно доработать итоговый вывод.
* Еще раз построй график "Ящик с усами" и проанализируй его.
* Рассмотри еще раз самые прибыльные жанры.
* Остальные рекомендации ты найдешь в моих комментариях!


**Желаю удачи и жду твой проект на повторное ревью!**😉

<div style="border:dashed green 1px; background:#097482; padding: 20px">
<span style="color:white">Мои комментарии V1:</span>

- c переводом в нижний регистр с помощью функции map я действительно перемудрил - надо просто работать с целой колонкой и pandas внутри все сделает
- по поводу библиотек согласен - я замотался и не прочитал секцию о настройке окружения. Сейчас все сделал по инструкции, создал новую среду в конде используя yaml файл предоставленный практикумом. Но там возникла проблема изза того, что я разработку веду сейчас на Маке M1 а для той не очень новой версии numpy, которая предоставлена - возникает ошибка сборки - я нашел на Stackoverflow описание проблемы. В latest версии numpy ее убрали и я могу инсталлировать numpy без проблем. Поэтому буду работать на самой новой версии у себя локально, потом загружать ее в практикум jupiter и еще раз там проверю и только после того как там уже не будет ошибок буду отправлять на ревью. Решение кривое но пока другого не придумал (у меня есть еще другой ноут на Win11 там проблем таких нет но работаю и на том и на другом ноуте). В текущей версии проекта я исправил ошибки которые ты указал и сейчас все в jupyter практикум запускается без ошибок
- по коэффициенту корреляции - добавил матрицы корреляции в разрезе каждой платформы и дал описание.
- расширил диаграммы для распределения по жанрам
- Subplots  я пробовал для размещения графиков в виде матрицы n х m - но там сильно съехали размеры и у меня не полчилось сделать красиво и не было времени разбираться - разберусь когда будет время
- Seaborn boxplot построил - все колонки в виде боксплотов на одном графике + плюс отрисовал дополнительно отдельно области с квартилями
